In [1]:
import optuna
import joblib
import datetime

/Users/kmenear/Projects/envSulfurTES/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from stesml.model_tools import build_train_test_model

/Users/kmenear/Projects/envSulfurTES/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [6]:
def objective(trial):   
    # Save the study before running the next trial
    joblib.dump(study, "../studies/study_" + model_type + "_" + datetime.datetime.now().strftime("%Y%m%d-%H") + ".pkl")
    
    if model_type == 'NN':
        scale = True
        n_layers = trial.suggest_int("n_layers", 1, 5)
        n_hidden_units = trial.suggest_int("n_hidden_units", 10, 100)
        batch_size = trial.suggest_int("batch_size", 10, 10000, log=True)
        parameters = {'n_layers': n_layers, 'n_hidden_units': n_hidden_units, 'batch_size': batch_size, 'epochs': 200}
        result, addendum = build_train_test_model(data_dir, model_type, target, metric, scale, parameters, n_repeats, t_max=360)
    elif model_type == 'XGBoost':
        scale = False
        learning_rate = trial.suggest_float("learning_rate", 0.01, 1)
        subsample = trial.suggest_float("subsample", 0.01, 1)
        colsample_bytree = trial.suggest_float("colsample_bytree", 0.01, 1)
        num_boost_round = 10000 # Set this as a maximum, model will stop with early stopping
        parameters = {'learning_rate': learning_rate, 'subsample': subsample, 'colsample_bytree': colsample_bytree, 'num_boost_round': num_boost_round}
        result, addendum = build_train_test_model(data_dir, model_type, target, metric, scale, parameters, n_repeats, t_min=360)
    elif model_type == 'RandomForest':
        scale = False
        n_estimators = trial.suggest_int("n_estimators", 1, 200, log=True)
        max_depth = trial.suggest_int("max_depth", 1, 100)
        max_samples = trial.suggest_float("max_samples", .01, 1, log=True)
        parameters = {'n_estimators': n_estimators, 'max_depth': max_depth, 'max_samples': max_samples}
        result, addendum = build_train_test_model(data_dir, model_type, target, metric, scale, parameters, n_repeats)
    
    return result

In [4]:
data_dir = "../data/Sulfur_Models/heating/heating_all"
model_type = 'XGBoost' # Options: NN, XGBoost, RandomForest
target = 'h' # Options: Tavg, h
metric = 'rmse' # Options: rmse, r2
n_repeats = 1 # Number of times to repeat 5-fold CV. Each repeat gives a different shuffle.

if metric == 'rmse':
    direction = 'minimize'
elif metric == 'r2':
    directon = 'maximize'
    
load_study = True
study_name = "study_NN_20220708-15.pkl"

In [5]:
if load_study:
    study = joblib.load("../studies/" + study_name)
    print("Best trial until now:")
    print(" Value: ", study.best_trial.value)
    print(" Params: ")
    for key, value in study.best_trial.params.items():
        print(f"    {key}: {value}")
else:
    study = optuna.create_study(direction=direction)

Best trial until now:
 Value:  0.30885909543570805
 Params: 
    n_layers: 1
    n_hidden_units: 82
    batch_size: 10


In [10]:
study.optimize(objective, n_trials=250)

[19:04:16] WARNING: ../src/learner.cc:576: 
Parameters: { "num_boost_round" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	test-rmse:36.08525
[20]	test-rmse:7.25067
[40]	test-rmse:7.49392
[45]	test-rmse:7.54717
Split #0, This Result: 7.5472, Average Result: 7.5472
[19:04:37] WARNING: ../src/learner.cc:576: 
Parameters: { "num_boost_round" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	test-rmse:31.49361
[20]	test-rmse:10.56465
[40]	test-rmse:9.00535
[60]	test-rmse:9.27064
[66]	test-rmse:9.30229
Split 

[I 2022-07-08 19:06:03,009] Trial 1 finished with value: 6.694686286377485 and parameters: {'learning_rate': 0.22039735498504276, 'subsample': 0.20661051170249276, 'colsample_bytree': 0.3394535618030828}. Best is trial 1 with value: 6.694686286377485.


Split #4, This Result: 4.3256, Average Result: 6.6947
[19:06:15] WARNING: ../src/learner.cc:576: 
Parameters: { "num_boost_round" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	test-rmse:15.15673
[20]	test-rmse:7.34032
[22]	test-rmse:7.25396
Split #0, This Result: 7.2540, Average Result: 7.2540
[19:06:37] WARNING: ../src/learner.cc:576: 
Parameters: { "num_boost_round" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	test-rmse:16.30763
[20]	test-rmse:5.80627
[31]	test-rmse:5.98935
Split #1, This Result:

[I 2022-07-08 19:08:08,451] Trial 2 finished with value: 5.524612415643093 and parameters: {'learning_rate': 0.7303137624182325, 'subsample': 0.7460404902385005, 'colsample_bytree': 0.8555578185452548}. Best is trial 2 with value: 5.524612415643093.


Split #4, This Result: 5.0932, Average Result: 5.5246
[19:08:20] WARNING: ../src/learner.cc:576: 
Parameters: { "num_boost_round" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	test-rmse:43.82908
[20]	test-rmse:25.02223
[40]	test-rmse:15.80593
[60]	test-rmse:11.68704
[80]	test-rmse:9.73049
[100]	test-rmse:8.67115
[120]	test-rmse:7.94514
[140]	test-rmse:7.60492
[160]	test-rmse:7.46429
[180]	test-rmse:7.19073
[200]	test-rmse:7.07516
[220]	test-rmse:6.97005
[240]	test-rmse:6.90494
[260]	test-rmse:6.92436
Split #0, This Result: 6.9265, Average Result: 6.9265
[19:09:15] WARNING: ../src/learner.cc:576: 
Parameters: { "num_boost_round" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  th

...

[I 2022-07-08 19:19:09,241] Trial 6 finished with value: 6.084889095493671 and parameters: {'learning_rate': 0.9965560493141323, 'subsample': 0.7118566668671396, 'colsample_bytree': 0.7585817884517483}. Best is trial 2 with value: 5.524612415643093.


Split #4, This Result: 7.4482, Average Result: 6.0849
[19:19:22] WARNING: ../src/learner.cc:576: 
Parameters: { "num_boost_round" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	test-rmse:23.19181
[20]	test-rmse:9.76637
[23]	test-rmse:9.32424
Split #0, This Result: 9.4351, Average Result: 9.4351
[19:19:42] WARNING: ../src/learner.cc:576: 
Parameters: { "num_boost_round" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	test-rmse:21.60832
[20]	test-rmse:9.04669
[40]	test-rmse:9.92543
[43]	test-rmse:9.84015

[I 2022-07-08 19:21:04,691] Trial 7 finished with value: 7.5000660606765335 and parameters: {'learning_rate': 0.5360431170621014, 'subsample': 0.6077393798054037, 'colsample_bytree': 0.2445039110923439}. Best is trial 2 with value: 5.524612415643093.


Split #4, This Result: 5.9115, Average Result: 7.5001
[19:21:17] WARNING: ../src/learner.cc:576: 
Parameters: { "num_boost_round" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	test-rmse:32.26539
[20]	test-rmse:8.18279
[29]	test-rmse:8.89785
Split #0, This Result: 8.8979, Average Result: 8.8979
[19:21:40] WARNING: ../src/learner.cc:576: 
Parameters: { "num_boost_round" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	test-rmse:28.44063
[20]	test-rmse:9.57957
[40]	test-rmse:9.62026
[45]	test-rmse:9.61449

KeyboardInterrupt: 

In [ ]:
best_params = study.best_params

In [18]:
best_params

{'n_layers': 2, 'n_hidden_units': 79, 'batch_size': 339, 'epochs': 7}

In [21]:
best_value = study.best_value

In [22]:
best_value

1.0137617092656621